# Hard Attention

I am interested in working through some of the mathematical formulation behind Hard Attention. I believe it is a helpful exercise, and I think the connections to reinforcement learning are intriguing. I will try to focus on aspects that I think are important, or that were helpful for me to understand. I reference the paper [*Show, Attend, and Tell*](https://arxiv.org/pdf/1502.03044.pdf) as well as our class notes.

We start with some basic setup that is fairly agnostic to the specific application.

$$ y = \{y_1,y_2,\ldots,y_C\},\:\:y_i\in\mathbb{R}^K $$
$$ a = \{a_1,a_2,\ldots,a_L\},\:\:a_i\in\mathbb{R}^D $$

The input is $a$ and the output is $y$. For example this could be an image and a caption.

We then have our attention variable which is a Multinoulli r.v. as it is a vector with elements either 1 or 0 (on or off).

$$ s_t = \{s_{t,1},s_{t,2},\ldots,s_{t,L}\} $$

Thus we either pay attention to the ith input or not, and we can determine this probability using the regular methods involving an attention function and softmax activations. We also note that $t$ indexes into the output y.

Importantly this is where soft attention differs from hard attention. In hard attention you take this "probability" (the output of the softmax) and treat it as a weighting for the input $a$. Thus we see that soft attention is stochastic (i.e. random) while hard attention is deterministic.

Our next step is to write a loss function. We know we are trying to model $p(y|a)$, so the start is easy...

$$ \log p(y|a) = \log(\sum_{s}p(s|a)p(y|s,a)) $$

Where this follows from total probability. Note that $s$ is a collection of $s_t$ as we are considering the whole output $y$. We then write the following to get our loss...

$$ \log\sum_{s}p(s|a)p(y|s,a) \geq \sum_{s}p(s|a)\log p(y|s,a) = L_s$$

Becuase $\log$ is a concave function this inequality is a direct result of Jensen's inequality where we treat our weights as $p(s|a)$.

Now we need to differentiate with respect to some arbitrary weights $W$ in our model.

$$ \frac{\partial L_s}{\partial W} = \sum_{s}p(s|a)\frac{\partial\log p(y|s,a)}{\partial W} + \log p(y|s,a)\frac{\partial p(s|a)}{\partial W} $$

Then we note the following:
$$ \frac{\partial \log f(x)}{\partial x} = \frac{f'(x)}{f(x)} $$

$$ \implies \frac{\partial L_s}{\partial W} = \sum_{s}p(s|a)\Big( \frac{\partial\log p(y|s,a)}{\partial W} +  \log p(y|s,a)\frac{\partial \log p(s|a)}{\partial W} \Big) $$

At this point a good question to ask might be why is this useful? The answer is that we can apply Monte Carlo sampling to approximate the gradient above which is necessary because we don't know the distribution $p(s|a)$. The use of Monte Carlo follows because we can treat the gradient as an expectation, so by sampling different values of $s$ we can determine the sample mean. In mathematical terms we can do the following:

$$ \frac{\partial L_s}{\partial W} \approx \frac{1}{N}\sum_{n=1}^N \frac{\partial\log p(y|\tilde{s}^n,a)}{\partial W} +  \log p(y|\tilde {s}^n,a)\frac{\partial \log p(\tilde{s}^n|a)}{\partial W} $$

The idea is that if we take $N$ large enough then our approximation will be good. It is also important to point out that we sample in the following way:

$$ \tilde{s}^n_t\sim Multinoulli(\alpha_t) $$

Where $\alpha_t$ is the output of our attention function inside our network for the given $a$ and $y$.

At this point I think the question is how does all of this relate to Reinforcement Learning? In short this is what we have been formulating all along. The method we produced above is (with some modifications) the REINFORCE learning rule, or at least so I am told. In RL we want to learn model weights using a system of reward for a given action. The better the model does at its job the higher reward it will get.

To see this we consider the following (which we only mentioned earlier):

$$ p(s_t|s_{i<t},a)=\alpha_t $$

The conditional portion of this probability can be seen as the current environment state. The attention information for previous words ($s_{i<t}$) and the input features ($a$) are what we need to make a new decision. The distribution of $\alpha_t$ is then our policy for making this decision. For example, a policy may be to attend to every feature.

So what is the reward for making a given decision? Well, during learning we want the decision to result in the ground-truth output $y$. Thus the reward is...

$$ p(y_t|s_{i\leq t},a) $$

This is saying given we chose (according to our policy) to attend in the manner prescribed by $s_t$, what is the probability we produce the correct output? This reward is a value between 0 and 1, with 0 indicating we never produce the right output and 1 indicating we always do. Thus we want to maximize the reward which is essentially maximizing the likelihood of producing the correct output.